Current problems
1. map() does not preserve the original dataset, but I want to keep the original in order to also create the diff train/test sets
2. Baseline and DA are reporting the same numbers

Current info

1a. Can map diff sets without affecting the other set

Potential solutions
1. At the very start when loading the data, load_dataset for all variations needed and apply appropriate map for each variation
  - This works and is what is currently implemented but feels inefficient?

TODO:
- [ ] Look at individual predictions bw baseline and da to determine if problem
is in evaluate or training

In [1]:
!pip install transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=a07bf1059c2b3d38a8ef88072789ca8817cc7eba9c516d06e7ceb915a5233516
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2

In [2]:
from datasets import load_dataset

conll = load_dataset('conll2003', trust_remote_code=True)
conll_da = load_dataset('conll2003', trust_remote_code=True)
conll_lower = load_dataset('conll2003', trust_remote_code=True)
conll_upper = load_dataset('conll2003', trust_remote_code=True)
label_list = conll['train'].features[f"ner_tags"].feature.names

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [3]:
def data_augmentation(example):
  example["tokens"] = example["tokens"] + [token.lower() for token in example["tokens"]] + [token.upper() for token in example["tokens"]]
  example["ner_tags"] = example["ner_tags"] + example["ner_tags"] + example["ner_tags"]
  return example

def lower_test(example):
  example["tokens"] = [token.lower() for token in example["tokens"]]
  return example

def upper_test(example):
  example["tokens"] = [token.upper() for token in example["tokens"]]
  return example

conll_da["train"] = conll_da["train"].map(data_augmentation)
conll_lower["test"] = conll["test"].map(lower_test)
conll_upper["test"] = conll["test"].map(upper_test)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}
{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.', 'eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.', 'EU', 'REJECTS', 'GERMAN', 'CALL', 'TO', 'BOYCOTT', 'BRITISH', 'LAMB', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0, 3, 0, 7, 0, 0, 0, 7, 0, 0, 3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [13]:
print(conll["train"][0])
print(conll_da["train"][0])
print(conll_lower["test"][10])
print(conll_upper["test"][10])

{'id': '10', 'tokens': ['takuya', 'takagi', 'scored', 'the', 'winner', 'in', 'the', '88th', 'minute', ',', 'rising', 'to', 'head', 'a', 'hiroshige', 'yanagimoto', 'cross', 'towards', 'the', 'syrian', 'goal', 'which', 'goalkeeper', 'salem', 'bitar', 'appeared', 'to', 'have', 'covered', 'but', 'then', 'allowed', 'to', 'slip', 'into', 'the', 'net', '.'], 'pos_tags': [22, 22, 38, 12, 21, 15, 12, 16, 21, 6, 39, 35, 37, 12, 22, 22, 37, 15, 12, 16, 21, 43, 38, 22, 22, 38, 35, 37, 40, 10, 30, 40, 35, 37, 15, 12, 21, 7], 'chunk_tags': [11, 12, 21, 11, 12, 13, 11, 12, 12, 0, 21, 22, 22, 11, 12, 12, 21, 13, 11, 12, 12, 11, 21, 11, 12, 21, 22, 22, 22, 0, 21, 22, 22, 22, 13, 11, 12, 0], 'ner_tags': [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 7, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': '10', 'tokens': ['TAKUYA', 'TAKAGI', 'SCORED', 'THE', 'WINNER', 'IN', 'THE', '88TH', 'MINUTE', ',', 'RISING', 'TO', 'HEAD', 'A', 'HIROSHIGE', 'YANAGIMOTO', 'CROSS', 'TOWARDS', 'THE',

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_conll = conll.map(tokenize_and_align_labels, batched=True)
tokenized_conll_da = conll_da.map(tokenize_and_align_labels, batched=True)
tokenized_conll_lower = conll_lower.map(tokenize_and_align_labels, batched=True)
tokenized_conll_upper = conll_upper.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

In [5]:
from transformers import DataCollatorForTokenClassification
import evaluate

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
seqeval = evaluate.load("seqeval")

In [6]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [7]:
id2label = {id: label for id, label in enumerate(label_list)}
label2id = {label:id for id, label in enumerate(label_list)}

In [8]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased", num_labels=9, id2label=id2label, label2id=label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args = TrainingArguments(
    output_dir="test_ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to = 'none',
)

baseline = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_conll["train"],
    eval_dataset=tokenized_conll["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

baseline.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.019400,0.119643,0.900122,0.912712,0.906374,0.982319
2,0.010900,0.130439,0.903416,0.922450,0.912834,0.982750


TrainOutput(global_step=1756, training_loss=0.013279577051045412, metrics={'train_runtime': 364.1003, 'train_samples_per_second': 77.127, 'train_steps_per_second': 4.823, 'total_flos': 701093358405576.0, 'train_loss': 0.013279577051045412, 'epoch': 2.0})

In [15]:
da_model = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_conll_da["train"],
    eval_dataset=tokenized_conll["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

da_model.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.080300,0.117033,0.894120,0.918024,0.905914,0.979800
2,0.022800,0.135235,0.889688,0.919618,0.904405,0.979498


TrainOutput(global_step=1756, training_loss=0.042760559529541296, metrics={'train_runtime': 991.4269, 'train_samples_per_second': 28.325, 'train_steps_per_second': 1.771, 'total_flos': 2960024288481576.0, 'train_loss': 0.042760559529541296, 'epoch': 2.0})

In [18]:
print("--------Lowercase Evaluation--------")
print(f"Baseline F1: {baseline.evaluate(eval_dataset=tokenized_conll_lower['test'])['eval_f1'] * 100:0.2f}")
print(f"DA F1: {da_model.evaluate(eval_dataset=tokenized_conll_lower['test'])['eval_f1'] * 100:0.2f}")
print("--------Uppercase Evaluation--------")
print(f"Baseline F1: {baseline.evaluate(eval_dataset=tokenized_conll_upper['test'])['eval_f1'] * 100:0.2f}")
print(f"DA F1: {da_model.evaluate(eval_dataset=tokenized_conll_upper['test'])['eval_f1'] * 100:0.2f}")

--------Lowercase Evaluation--------


Baseline F1: 48.51


DA F1: 48.51
--------Uppercase Evaluation--------
Baseline F1: 50.22
DA F1: 50.22
